In [21]:
import sys
import random
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import networkx as nx

from copy import deepcopy
from typing import Union, List, Tuple, Dict
from pgmpy.readwrite import XMLBIFReader

from BayesNet import *
from BNReasoner import *
from utils import *

# Initialize
---

In [8]:
# BIFXML files
net = './testing/lecture_example.BIFXML'

In [9]:
# constructs a BN object
bn = BayesNet()

# Loads the BN from an BIFXML file
bn.load_from_bifxml(net)

# create reasoner
bnr = BNReasoner(bn)

In [38]:
# query inputs

evidence = {'Winter?': True}

x = ['Slippery Road?']

prio = ['Wet Grass?','Rain?']

# Usage
---

### D-separation
Checks whether x and y are d-separated, given (possibly empty) evidence. 

* x and y must be lists
* evidence is (always) a dict
* output is boolean

In [39]:
dsep = bnr.d_separated(prio, x, evidence)
print(f"Are {prio} and {x} d-separated by evidence '{next(iter(evidence))} = {evidence[next(iter(evidence))]}'?\n\n{dsep}")

Are ['Wet Grass?', 'Rain?'] and ['Slippery Road?'] d-separated by evidence 'Winter? = True'?

True


### Ordering functions for determining elimination order
inputs should be network structure
* y = list of query variables
* z = priority variables which are ordered first __[optional]__

In [48]:
pi1 = mindeg_order(net=bn, query=prio)
pi2 = minfill_order(net=bn, query=prio)
print(f"Variable orderings mindeg:\t{pi1}")
print(f"Variable orderings minfill:\t{pi2}")

Variable orderings mindeg:	['Wet Grass?', 'Rain?']
Variable orderings minfill:	['Wet Grass?', 'Rain?']


### Marginal distributions
with optional ordering function (name of function is input)

In [57]:
# without evidence:
prior_marginal = bnr.compute_marginal(query=x, ordering_function=random_order)
print(f"\n\nPrior Marginal for Query {x}\n{prior_marginal}")

# with evidence:
posterior_marginal = bnr.compute_marginal(query=x, evidence=evidence, ordering_function=minfill_order)
print(f"\nPosterior Marginal for Query {x} with Evidence {evidence}\n{posterior_marginal}")



Prior Marginal for Query ['Slippery Road?']
{'f3':    Slippery Road?      p
0           False  0.636
1            True  0.364}

Posterior Marginal for Query ['Slippery Road?'] with Evidence {'Winter?': True}
{'f3':    Slippery Road?      p
0           False  0.264
1            True  0.336}


### Most likely instantiations

In [59]:
# MAP:
MAP = bnr.MAP(query=x, evidence=evidence)
print(f"MAP:\n{[MAP[key] for key in MAP]}\n")

# MPE:
MPE = bnr.MPE(evidence=evidence, ordering_function=minfill_order)
print(f"MPE:\n{[MPE[key] for key in MPE]}")

MAP:
[p                      0.336
inst_Slippery Road?     True
Name: 1, dtype: object]

MPE:
[p                      0.21504
inst_Rain?                True
inst_Slippery Road?       True
Name: 1, dtype: object]
